# Importador de Resultados da Megasena

Para fins educativos, eu criei um pequeno notebook que importa os dados dos jogos da megasena a partir do site da caixa (https://loterias.caixa.gov.br/Paginas/Download-Resultados.aspx).<br>
Não preciso dizer que o código é apresentado como é sem nenhuma garantia ou reponsabilidade de uso do mesmo.


In [ ]:
import pandas as pd
import urllib.request
import ssl
import json
from bs4 import BeautifulSoup

A Função abaixo serve para remover os espaços e quebras de linhas das celulas do BeautifulSoup.
Com certeza deve haver um jeito mais elegante de fazer

In [ ]:
#
# Remove espaços e quebras de linha da celular do BeautifulSoup
#
def stripText(a):
    if a:
        return ''.join(a.text.split())

In [ ]:
#
# Faz a requisição HTTP para o site da caixa
# 
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode    = ssl.CERT_NONE
#
# URL Abaixo é a que rendeiza a tabela apresentada no link: https://loterias.caixa.gov.br/Paginas/Download-Resultados.aspx
#
url = 'https://servicebus2.caixa.gov.br/portaldeloterias/api/resultados?modalidade=Mega-Sena'
req = urllib.request.Request(url=url)
f   = urllib.request.urlopen(req,context=ctx)
js  = f.read().decode('utf-8')
js  = json.loads(js)


In [ ]:
#
# Aqui a gente pega o HTML que veio no JSON da caixa e intepreta ele como HTML :)
#
soup = BeautifulSoup(js['html'])


### Sobre a estrutura de dados
A estrutura de dados que é retornada pela tabela é um tanto quanto quanto confusa, com tabelas dentro de tabelas, então o código abaixo lida com essa complexidade para tentar extrair de forma simples as colunas que são renderizadas no browser.<br>
No final desta célula , teremos uma variável 'head', com a lista dos campos e uma variável data, com as linhas e os valores dos campos.

In [ ]:
#
# Extrai os resultados da Megasena do Site da Caixa :)
#
table  = soup.find('table')
#
# head vai conter a lista de colunas com nomes :)
#
head   = table.find_all('thead',recursive=False)
head   = head[0].findChildren('tr',recursive=False)
head   = head[0].findChildren('th',recursive=False)
head   = list(map(stripText,head))
bodies = table.find_all('tbody',recursive=False)
#
# data, é uma lista de de listas com todas as colunas
#
data   = []
for body in bodies:
    a     = body.find('tr',recursive=False)
    cells = a.findChildren('td',recursive=False)
    row_data  = []
    for cell in cells:
        text = cell.text
        text = ''.join(text.split())
        row_data.append(text)
    data.append(row_data)
    

### Resultado
Abaixo você pode ver a lista de resultados dos jogos imprimida na ordem, note que eu não fiz nenhum tratamento com relação ao tipo de dado, mas eu acho que serve como uma ponto de partida para seus estudos.
Ah sim!, eu só imprimi as colunas dos resultados, mas poderia ser impresso tudo.

In [ ]:
#
# Imprimi todos os resultados por sorteio
#
for result in data:
 
    #
    # Imprimi apenas os resultados dos jogos.
    #
    print(result[0],result[2:8])
    #
    # Imprimi todos os dados da tabela
    #
    # print(result[0],result)